In [2]:
from bs4 import BeautifulSoup
import requests
import re
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from datetime import datetime, timedelta
import time
import os
import pandas as pd

In [ ]:
all_news_links = []
    
url = 'https://www.dnaindia.com/search?q=bangladesh'


base_url = "https://www.firstpost.com/"
country = 'bangladesh'
initial_url = f'{base_url}search?q={country}#gsc.tab=0&gsc.q=bangladesh&gsc.sort=date'

# chrome_options = Options()
# chrome_options.add_argument("--headless")
# driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
driver.get(url)

#this Website displays news articles with the help of Google/enchanced by Google.
#So, each page needs to be separately selected and visited to scrape the links. 
#that can't be done by changing the Page Number in the URl. So, it's done using ActionsChains 'Click' function.

#the links are then filtered to remove unnecessary news, since the search consists of any and all news that 
#remotely mention 'bangaldesh', even if it isn't strongly related to Bangladesh. 

#The first 5 pages are taken here, so 50 news links. That's because the news aren't sorted in order, and there are some news from 
#long ago that are present in the first page and some recent news present in later pages. 
#There are also multiple news links which lead to a 404 Error Page. 

for page in range(1,6):
    
    try:
        buttons = driver.find_elements(By.CLASS_NAME, 'gsc-cursor-page')
        
        for button in buttons: 
           
            if button.get_attribute('innerHTML') == f'{page}':
                print(f'in page {page}')
                ActionChains(driver)\
                .click(button)\
                .perform()
                time.sleep(20)                  

                main_div = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div/div[2]/div/div/div/div/div[5]/div[2]/div[1]/div/div[1]')
     
                if main_div:
                    print('main_div found')
                    time.sleep(20)
                    news = main_div.find_elements(By.CSS_SELECTOR, 'a')

                    for n in news:
                        news_text = n.get_attribute('data-ctorig')

                        if news_text in all_news_links:
                            continue
                        else:
                            if 'bangladesh' in news_text or 'bangladeshs' in news_text:              
                                print(news_text)
                                all_news_links.append(news_text)
                            else:
                                continue     
    except:
       
            pass

driver.quit()

In [ ]:
len(all_news_links)

In [6]:
all_news_links = []

base_url = "https://www.dnaindia.com/"
country = 'bangladesh'
initial_url = f'{base_url}search?q={country}'

# chrome_options = Options()
# chrome_options.add_argument("--headless")
# driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
driver.get(initial_url)

#this Website displays news articles with the help of Google/enchanced by Google.
#So, each page needs to be separately selected and visited to scrape the links. 
#that can't be done by changing the Page Number in the URl. So, it's done using ActionsChains 'Click' function.

#the links are then filtered to remove unnecessary news, since the search consists of any and all news that 
#remotely mention 'bangaldesh', even if it isn't strongly related to Bangladesh. 

#The first 5 pages are taken here, so 50 news links. That's because the news aren't sorted in order, and there are some news from 
#long ago that are present in the first page and some recent news present in later pages. 
#There are also multiple news links which lead to a 404 Error Page. 

#WHILE LOOP CAN BE USED FOR FINDING TOTAL NUMBER OF PAGES. 2 PAGES TAKEN HERE 

for page in range(1,3):
    
    try:
        buttons = driver.find_elements(By.CLASS_NAME, 'gsc-cursor-page')
        
        for button in buttons: 
           
            if button.get_attribute('innerHTML') == f'{page}':
                print(f'in page {page}')
                ActionChains(driver)\
                .click(button)\
                .perform()
                time.sleep(20)                  

                main_div = driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div/div[2]/div/div/div/div/div[5]/div[2]/div[1]/div/div[1]')
     
                if main_div:
                    print('main_div found')
                    time.sleep(10)
                    news = main_div.find_elements(By.CSS_SELECTOR, 'a')

                    for n in news:
                        news_text = n.get_attribute('data-ctorig')

                        if news_text in all_news_links:
                            continue
                        else:
                            if 'bangladesh' in news_text or 'bangladeshs' in news_text:              
#                                 print(news_text)
                                all_news_links.append(news_text)
                            else:
                                continue     
    except:
       
            pass

driver.quit()


data_list = []
counter = 0
for link in all_news_links:
    

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    #driver = webdriver.Chrome()
    driver.get(link)

    country = 'India'

    try:

        title_tag = driver.find_element(By.CLASS_NAME, 'article-heading')
        title = title_tag.get_attribute('innerHTML')
    except:
        title = 'Title Not Found'


    try:
        author_main_tag = driver.find_element(By.ID, 'dna-home')
        author_tag = author_main_tag.find_element(By.CSS_SELECTOR, 'a')
        author = author_tag.get_attribute('innerHTML')

    except:
        author = 'Author Not Found'

    #The Date Info is available in different formats for different news articles. It's a bit difficult to match format for all
    #articles. But, the functions below should match most articles.
    #I was not able to find the Date Info in the Meta Tags. 
    
    #initializing here. if these stay blank, the news link will be automatically eliminated.
    
    source_localtime = ''
    bangladesh_localtime = ''
    
    try:  
        date_main_tag = driver.find_element(By.CLASS_NAME, 'dna-update')
        date_data = date_main_tag.get_attribute('innerHTML')
    
    except:
                
        try:
            date_main_tag = driver.find_element(By.CLASS_NAME, 'article-author-txt')
            date_data = date_main_tag.get_attribute('innerHTML').split('Updated: ')[1]
            
        except NoSuchElementException:
            date_data = 'Date Data Not Found'

    if date_data!='Date Data not found':
        
        if '>' in date_data:
            
            only_date = date_data.split('>')[1]
        
        else:
            
            only_date = date_data

        try:
            
#             format_change = datetime.strptime(only_date,'%b %d, %Y, %I:%M %p IST') + timedelta(minutes = 30)
            
            source_localtime = datetime.strptime(only_date, "%b %d, %Y, %I:%M %p IST")
            bangladesh_localtime = source_localtime + timedelta(minutes=30)
            

        
        except ValueError:
            
            date_data = 'Date Data Not Found'    


    #CHAGING TO BEAUTIFUL SOUP 

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    content_div = soup.find('div', class_='article-description')

    if content_div:
    
    #the content is inside <p> tags, and in some websites, they are also inside <ul> tags. 
    #so, here, we try to fin all <p> and <ul> tags. However, not all <ul> tags represent actual content.
    #The <ul> tags that represent News Content don't have any particular Class Name, so we're taking all <ul> content
    #that don't have a class name. It has been working correctly so far. 
    
        paragraph_tags = content_div.find_all(True)
        content = []
    
        for tag in paragraph_tags:

            if tag.name == 'p':

                content.append(tag.text)

            elif tag.name == 'ul':        
                if not tag.has_attr('class'):
                    content.append(tag.text)
            else:
                continue 

        full_content = ' '.join(content)
        full_content = full_content.split('READ')[0]
        
    else:
        full_content = 'Content Not Found'


    content_summary_section = soup.find('section',class_ ='article-details')
    
    if content_summary_section:
        
        content_summary_tag = content_summary_section.find('p', class_ = 'article-short-desc')
        content_summary = content_summary_tag.text if content_summary_tag else 'Content summary not found'
        
    title_translation = 'None'
    summary_translation = 'None'
    content_translation = 'None'

#     data_dict = {
#         "country": country,
#         "author": author,
#         "url": link,
#         "title": title,
#         "timestamp": timestamp,
#         "date": date,
#         "date_str": date_str,
#         "content": full_content,
#         "content_summary": content_summary
#     }
    
    data_dict = {
            "url": link,
            "title": title,
            "content": full_content,
            "content_summary": content_summary,
            "title_translation":title_translation,
            "content_translation":content_translation,
            "summary translation":summary_translation,
            "author": author,
            "country": country,
            'source_localtime': source_localtime,
            'bangladesh_localtime': bangladesh_localtime

        }
    
    
    print(counter)
    counter+=1


    if (date_data != "Date Data Not Found" and full_content != "Content Not Found" and content_summary != "Content summary not found"):
        if data_dict not in data_list:
            # Adding to data list
            data_list.append(data_dict)
            print(f'Link {counter} added')
    else:
        print(counter)
        print('Skipped due to missing info.')

df = pd.DataFrame(data_list)
df.head()

in page 1
main_div found
in page 2
main_div found
0
1
Skipped due to missing info.
1
2
Skipped due to missing info.
2
3
Skipped due to missing info.
3
4
Skipped due to missing info.
4
5
Skipped due to missing info.


KeyboardInterrupt: 

In [ ]:
country = 'India'
csv_filename = f"{country}_DNA_India.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["date"] = pd.to_datetime(df["date"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="date", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df["date"] = pd.to_datetime(df["date"], format = "%d-%m-%Y")  # Converting the "date" column to datetime
    df = df.sort_values(by="date", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)

In [7]:
all_news_links

['https://www.dnaindia.com/analysis/report-dna-tv-show-why-were-elections-in-bangladesh-controversial-this-year-awami-league-sheikh-hasina-sourabh-jain-3073910',
 'https://www.dnaindia.com/explainer/report-dna-explainer-what-is-the-water-dispute-between-india-and-bangladesh-know-purpose-of-kushiyara-river-pact-2983049',
 'https://www.dnaindia.com/cricket/report-meet-umme-ahmed-shishir-wife-of-bangladesh-captain-shakib-al-hasan-know-their-beautiful-love-story-3066787',
 'https://www.dnaindia.com/sports/report-rare-incident-on-cricket-field-bangladesh-recalls-ish-sodhi-after-controversial-runout-hasan-mahmud-liton-das-3062042',
 'https://www.dnaindia.com/cricket/bangladesh-vs-sri-lanka-live-score-world-cup-2023-eliminated-bangladesh-playing-for-pride-against-sri-lanka-3067325',
 'https://www.dnaindia.com/cricket/report-champions-trophy-2025-qualification-scenarios-for-bangladesh-netherlands-3068110',
 'https://www.dnaindia.com/world/report-bangladesh-polls-sheikh-hasina-s-party-manifesto

In [47]:
url = 'https://www.dnaindia.com/cricket/bangladesh-vs-sri-lanka-live-score-world-cup-2023-eliminated-bangladesh-playing-for-pride-against-sri-lanka-3067325'
url_2 = 'https://www.dnaindia.com/explainer/report-dna-explainer-what-is-the-water-dispute-between-india-and-bangladesh-know-purpose-of-kushiyara-river-pact-2983049'
# for link in all_news_links[:15]:
    
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
print(response.status_code)

date_main_tag = soup.find('p', class_ = 'dna-update')

if date_main_tag:
    
    date_data = date_main_tag.text
    date_data = date_data.split(': ',1)[1]

    remove_IST_and_others = date_data.split(' IST | ')

    if len(remove_IST_and_others) > 1:

        date_data = remove_IST_and_others[0]

    else:

        date_data = remove_IST_and_others[0].split(' IST ')[0]
        
else:
    
    alt_tag = soup.find('div', class_ = 'article-author-txt')
    
    alt_date = alt_tag.text.split('Updated: ')[1]
    
    remove_IST_and_others = alt_date.split(' IST')

    if len(remove_IST_and_others) > 1:

        date_data = remove_IST_and_others[0]

    else:

        date_data = remove_IST_and_others[0].split(' IST ')[0]


date_data


403


AttributeError: 'NoneType' object has no attribute 'text'

In [44]:
x = 'Reported By:| Edited By: DNA  Web Team |Source:  |Updated: Nov 06, 2023, 10:01 PM IST'

x.split('Updated: ')[1]

'Nov 06, 2023, 10:01 PM IST'